In [1]:
!pip install PyDrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"1YX9W3ARoWpTHE1zXp2hid08Tqd5OVIV-"})
downloaded.GetContentFile('splitTrainVal.zip')

In [ ]:
downloaded = drive.CreateFile({'id':"1iVrm_knNMMLZM6T4-f6GaIjBFUikWz28"})
downloaded.GetContentFile('damagePartClassification.py')

In [ ]:
downloaded = drive.CreateFile({'id':"1Pm6iVPZrD8czfUNCq29NGRx0WaMVOc1j"})
downloaded.GetContentFile('precisionAndRecall.py')

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
!pip install split_folders

In [ ]:
!ls /content/gdrive/My\ Drive/colab/carPartDamage/carPartDamageDetection/Models

ls: cannot access '/content/gdrive/My Drive/colab/carPartDamage/carPartDamageDetection/Models': No such file or directory


In [34]:
import os
import torch
import torch.utils.data
import torchvision
from PIL import Image
from pycocotools.coco import COCO
import numpy as np
import cv2
import time

In [2]:
class myOwnDataset(torch.utils.data.Dataset):
    def __init__(self, root, annotation, transforms=None):
        self.root = root
        self.transforms = transforms
        self.coco = COCO(annotation)
        self.ids = list(sorted(self.coco.imgs.keys()))

    def __getitem__(self, index):
        # Own coco file
        coco = self.coco
        # Image ID
        img_id = self.ids[index]
        # List: get annotation id from coco
        ann_ids = coco.getAnnIds(imgIds=img_id)
        # Dictionary: target coco_annotation file for an image
        coco_annotation = coco.loadAnns(ann_ids)
        # path for input image
        path = coco.loadImgs(img_id)[0]['file_name']
        #path = coco.loadImgs(img_id)[0]['/content/sample_data/my_data']
        # open the input image
        img = Image.open(os.path.join(self.root, path))

        # number of objects in the image
        num_objs = len(coco_annotation)

        # Bounding boxes for objects
        # In coco format, bbox = [xmin, ymin, width, height]
        # In pytorch, the input should be [xmin, ymin, xmax, ymax]
        boxes = []
        for i in range(num_objs):
            xmin = coco_annotation[i]['bbox'][0]
            ymin = coco_annotation[i]['bbox'][1]
            xmax = xmin + coco_annotation[i]['bbox'][2]
            ymax = ymin + coco_annotation[i]['bbox'][3]
            boxes.append([xmin, ymin, xmax, ymax])
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # Labels (In my case, I only one class: target class or background)
        labels = torch.ones((num_objs,), dtype=torch.int64)
        print(num_objs)
        print(labels)
        # Tensorise img_id
        img_id = torch.tensor([img_id])
        # Size of bbox (Rectangular)
        areas = []
        for i in range(num_objs):
            areas.append(coco_annotation[i]['area'])
        areas = torch.as_tensor(areas, dtype=torch.float32)
        # Iscrowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        # Annotation is in dictionary format
        my_annotation = {}
        my_annotation["boxes"] = boxes
        my_annotation["labels"] = labels
        my_annotation["image_id"] = img_id
        my_annotation["area"] = areas
        my_annotation["iscrowd"] = iscrowd

        if self.transforms is not None:
            img = self.transforms(img)

        return img, my_annotation

    def __len__(self):
        return len(self.ids)

In [3]:
# In my case, just added ToTensor
def get_transform():
    custom_transforms = []
    custom_transforms.append(torchvision.transforms.ToTensor())
    return torchvision.transforms.Compose(custom_transforms)


In [4]:
# path to your own data and coco file
train_data_dir = '/content/sample_data/my_data'
train_coco = '/content/sample_data/json_data/instances_default.json'


In [17]:
# create own Dataset
my_dataset = myOwnDataset(root=train_data_dir,
                          annotation=train_coco,
                          transforms=get_transform()
                          )

# collate_fn needs for batch
def collate_fn(batch):
    return tuple(zip(*batch))

# Batch size
train_batch_size = 1
print('Number of samples: ', len(my_dataset))

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
Number of samples:  325


In [23]:
# split the dataset in train and test set - using 80% for training, 20% for validation
indices = torch.randperm(len(my_dataset)).tolist()
train_dataset = torch.utils.data.Subset(my_dataset, indices[:-50])
valid_dataset = torch.utils.data.Subset(my_dataset, indices[-50:])

In [24]:
# own DataLoader
data_loader_train = torch.utils.data.DataLoader(train_dataset,
                                          batch_size=train_batch_size,
                                          shuffle=True,
                                          num_workers=4,
                                          collate_fn=collate_fn)

In [25]:
# own DataLoader
data_loader_val = torch.utils.data.DataLoader(valid_dataset,
                                          batch_size=train_batch_size,
                                          shuffle=True,
                                          num_workers=4,
                                          collate_fn=collate_fn)

In [26]:
len(data_loader_val)

50

In [27]:
len(data_loader_train)

275

In [28]:
# select device (whether GPU or CPU)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# DataLoader is iterable over Dataset
for imgs, annotations in data_loader:
    imgs = list(img.to(device) for img in imgs)
    annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
    print(annotations)

1
tensor([1])
0
2
2
tensor([], dtype=torch.int64)
tensor([1, 1])
tensor([1, 1])
1
tensor([1])
2
2
tensor([1, 1])
tensor([1, 1])
2
tensor([1, 1])
2
0
tensor([1, 1])
2
tensor([], dtype=torch.int64)
1
tensor([1, 1])
tensor([1])
1
1
2
tensor([1])
tensor([1])
4
tensor([1, 1])
tensor([1, 1, 1, 1])
[{'boxes': tensor([[728.3600, 176.5200, 754.7000, 212.9100]], device='cuda:0'), 'labels': tensor([1], device='cuda:0'), 'image_id': tensor([287], device='cuda:0'), 'area': tensor([958.5126], device='cuda:0'), 'iscrowd': tensor([0], device='cuda:0')}]
[{'boxes': tensor([[154.7000, 434.7000, 221.1000, 490.1800],
        [744.7000, 293.8000, 773.8200, 323.8200]], device='cuda:0'), 'labels': tensor([1, 1], device='cuda:0'), 'image_id': tensor([167], device='cuda:0'), 'area': tensor([3683.8721,  874.1824], device='cuda:0'), 'iscrowd': tensor([0, 0], device='cuda:0')}]
[{'boxes': tensor([], device='cuda:0'), 'labels': tensor([], device='cuda:0', dtype=torch.int64), 'image_id': tensor([63], device='cuda:0

In [29]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=False)
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model
    

# 2 classes; Only target class or background
num_classes = 2
#num_classes = 7
num_epochs = 10
model = get_model_instance_segmentation(num_classes)

# move model to the right device
model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256)
          (relu): ReLU(inplace=True)
          (downsample)

In [35]:
# parameters
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

len_dataloader_train = len(data_loader_train)

itr = 1
total_train_loss = []
        
for epoch in range(num_epochs):
    start_time = time.time()
    train_loss = []
   # model.train()
   
    for imgs, annotations in data_loader_train:
      #  i += 1
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        #Forward propagation
        loss_dict = model(imgs, annotations)
        losses = sum(loss for loss in loss_dict.values())
        #Reseting Gradients
        optimizer.zero_grad()
        #backward propagation
        losses.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        
        itr += 1

        print(f'Iteration: {i}/{len_dataloader_train}, Loss: {losses}')
        epoch_train_loss = np.mean(train_loss)
        total_train_loss.append(epoch_train_loss)
        print(f'Epoch train loss is {epoch_train_loss:.4f}')
        time_elapsed = time.time() - start_time
        print("Time elapsed: ",time_elapsed)
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': epoch_train_loss
            }, "checkpoint.pth")

1
tensor([1])
2
0
2
tensor([1, 1])
tensor([], dtype=torch.int64)
tensor([1, 1])
1
2
tensor([1, 1])
tensor([1])
2
1
tensor([1, 1])
tensor([1])


ValueError: ignored

In [ ]:
itr = 1
total_valid_loss = []
        
for epoch in range(num_epochs):
    start_time = time.time()
    valid_loss = []
   # model.train()
   
    for imgs, annotations in data_loader_valid:
      #  i += 1
        imgs = list(img.to(device) for img in imgs)
        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
        #Forward propagation
        loss_dict = model(imgs, annotations)
        losses = sum(loss for loss in loss_dict.values())
        #Reseting Gradients
        optimizer.zero_grad()
        #backward propagation
        losses.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        
        itr += 1

        print(f'Iteration: {i}/{len_dataloader_valid}, Loss: {losses}')
        epoch_valid_loss = np.mean(valid_loss)
        total_valid_loss.append(epoch_valid_loss)
        print(f'Epoch train loss is {epoch_valid_loss:.4f}')
        time_elapsed = time.time() - start_time
        print("Time elapsed: ",time_elapsed)
        